In [1]:
%load_ext autoreload
%autoreload 2

import os
import pickle
import pandas as pd
import plotly.io as pio

""" load data """
with open(os.path.join('..', 'dataset', 'voxel_maps.pkl'), "rb") as voxel_maps_file:
    voxel_maps = pickle.load(voxel_maps_file)

# display options
pd.set_option('display.max_columns', None)
pd.set_option('display.width', 100000)
pio.renderers.default = "plotly_mimetype+notebook_connected"

# Explore the 3D Reconstruction Interactively

Here we show a small selection of 3D reconstructions recorded in our wind tunnel experiments.
First we highlight two different experiments, Experiment_8 and Experiment_7. Both contain a scale model of a landscape:
While Experiment_8 contains the exact "Setup C" shown below, for Experiment_7 it is rotated by 45° CCW, thus changing the angle of attack of the wind. This is referred to as "Setup D".
##### Scale Model Landscape in Setup C
<div>
<img src="setupC.png" width="300"/>
</div>

In [2]:
import plotly.graph_objects as go
import numpy as np

def plot3d(exp_idx, sensor_type, column):
    setup = voxel_maps[exp_idx]['experiment_info']['setup']

    xx = voxel_maps[exp_idx]['voxel_map']['x']
    yy = voxel_maps[exp_idx]['voxel_map']['y']
    zz = voxel_maps[exp_idx]['voxel_map']['z']
    val = voxel_maps[exp_idx]['voxel_map'][sensor_type][column]

    val = np.nan_to_num(val, nan=0.001)

    fig = go.Figure(data=[
        go.Scatter3d(
        x=xx, y=yy, z=zz,
        mode='markers',
        marker=dict(
            size=np.maximum(75*val/np.max(val), 3),
            color=val,
            colorscale='Turbo',
            opacity=0.5,
            colorbar=dict(title='ppm'), # define colorbar
            showscale=True              # enable colorbar
        ),
    )])


    factor = 0.6
    fig.update_layout(
        title=f"Setup {setup} – Showing {sensor_type} {column} in Experiment_{exp_idx+1}",
        template='plotly_white',
        height=700,
        scene=dict(
            xaxis=dict(range=[-1.2, 1.2]),
            yaxis=dict(range=[-1.2, 1.2]),
            zaxis=dict(range=[0, 1.5]),
            aspectmode='manual',
            aspectratio=dict(x=2.4*factor, y=2.4*factor, z=1.5*factor),
            camera=dict(
                eye=dict(x=2, y=0.6, z=0.3),     # Position of the camera
            ),
        ),
    )

    fig.show()

In [3]:
plot3d(exp_idx=7, sensor_type='PID-sensor', column='ppm_relative')

In [4]:
plot3d(exp_idx=6, sensor_type='PID-sensor', column='ppm_relative')

## PID Senor vs MiCS Metal Oxide Sensors
Now we continue with Experiment_3 and show the output signals for the other sensors mounted on the sensor platforms.
The previous plots showed the PID sensor signal. Below, we show the MiCS 5524 and the MiCS 6814 (NH3 channel).

In every plot here, the output is of the column `ppm_relative`, which is the end product of our preprocessing chain: We derive a background-subtracted reading converted into ppm-equivalents of the target gasses. This removes the background drift over the course of the experiments. The sensor samples at 10Hz and the measurements are averaged to generate the scatterplots. For Experiments in setup C and D, the sensor samples for 5s; in Experiment 3 (Setup B), sensors sample for 20s.

In [5]:
plot3d(exp_idx=2, sensor_type='MiCS5524', column='ppm_relative')

In [6]:
plot3d(exp_idx=2, sensor_type='MiCS6814_NH3', column='ppm_relative')

## Wind Vectors
The same can be done with the wind data:

In [7]:
exp_idx = 7
xx = voxel_maps[exp_idx]['voxel_map']['x']
yy = voxel_maps[exp_idx]['voxel_map']['y']
zz = voxel_maps[exp_idx]['voxel_map']['z']
uu = voxel_maps[exp_idx]['voxel_map']['wind-u']
vv = voxel_maps[exp_idx]['voxel_map']['wind-v']
ww = voxel_maps[exp_idx]['voxel_map']['wind-w']

fig = go.Figure(data = go.Cone(
    x=xx,
    y=yy,
    z=zz,
    u=uu,
    v=vv,
    w=ww,
    colorscale='Portland',
    sizemode="scaled",
    sizeref=1,
    colorbar=dict(
        title="Wind Speed in m/s",
    ),
))

setup = 'C'
factor = 0.6
fig.update_layout(
    title=f"Setup {setup} – Showing Wind Vectors in Experiment_{exp_idx+1}",
    template='plotly_white',
    height=700,
    scene=dict(
        xaxis=dict(range=[-1.2, 1.2]),
        yaxis=dict(range=[-1.2, 1.2]),
        zaxis=dict(range=[0, 1.5]),
        aspectmode='manual',
        aspectratio=dict(x=2.4*factor, y=2.4*factor, z=1.5*factor),
        camera=dict(
            eye=dict(x=1, y=2, z=0.5),     # Position of the camera
        ),
    )
)

fig.show()

Explore the field by pressing your left, middle or right mouse button and dragging the mouse to rotate, zoom or translate.

When you zoom in on the locations of the obstacles, i.e. the buildings in the scale model landscape, you can make out how the wind is slowed down and diverted around the obstacles.
The overall vertial wind profile is also visible from the colormap.

Next, we can look at a slice of this data in an XZ plane:

In [8]:
import plotly.figure_factory as ff

slice_y = -0.54
voxel_map = voxel_maps[exp_idx]['voxel_map']
slice = voxel_map[voxel_map['y'] == slice_y]

xx = slice['x']
zz = slice['z']
uu = slice['wind-u']
ww = slice['wind-w']

fig = ff.create_quiver(
    xx, zz, uu, ww,
    scaleratio=1,
    scale=0.08,
)

fig.update_layout(
    title=f"Setup {setup}, sliced at y={slice_y}m – Showing Wind Vectors in Experiment_{exp_idx+1}",
    template='plotly_white',
    height=700,
    xaxis_title="x in m",
    yaxis_title="z in m",
)
fig.show()

What is especially striking here is how the wind is slowed down in front of obstacles or diverted around it sideways (in Y direction, into the figure plane),
and how it is curling downwards behind an obstacle, with air rushing down to fill the void left in the wake.
In some places (here at x=-0.65m), the wind vector even curls back completely, obtaining a small positive x component.

# VTK Files
All data presented here is also contained in the `.vtk` files that you find in the dataset.
Using those, users can also explore the spatial data using an application like ParaView.